# Fully leafed subtree from a tree

On se place dans le cas particulier dans lequel le graphe est un arbre a n sommets, et on cherche parmis les sous-arbre de taille k celui qui a le plus de feuilles.

On veux generer le programme lineaire a partir d'un graphe.

## Le programme linéaire

### Variables

$$ 
x_{i,j} =
  \begin{cases}
    0, & \text{Absence d'arête entre les sommets i et j dans le sous-arbre} \\
    1, & \text{Présence d'arête entre les sommets i et j dans le sous-arbre}
  \end{cases}
$$
 
$$ 
y_{i} =
  \begin{cases}
    0, & \text{Absence du sommet i dans le sous-arbre} \\
    1, & \text{Présence du sommet i dans le sous-arbre}
  \end{cases}
$$
 
$$ 
f_{i} =
  \begin{cases}
    0, & \text{Le sommet i n'est pas une feuille} \\
    1, & \text{Le sommet i est une feuille}
  \end{cases}
$$



### Fonction objectif

On souhaite maximiser la fonction suivante: 
$$ f() = \sum_{0}^{i} f_{i} $$

### Contraintes

On impose la taille du sous-arbre : 
$$ \sum_{0}^{i} y_{i} = k $$

On vérifie que le nombre d'arêtes est égal au nombre de sommets - 1 :
$$ \sum_{}^{} x_{i,j} = k - 1 $$

Présence d'une arête : 
$$ x_{i,j} <= \frac{y_{i} + y_{j}}{2} $$

Une feuille est un sommet (mais pas forcément l'inverse) : 
$$ f_{i} <= y_{i} $$

Une feuille est un sommet de degré 1 :
$$ f_{i} <= 1 + \frac{1 - d}{m} $$ avec m le degré maximal du graphe

In [11]:
def generate_program(g, k):
# Resolution du probleme pour un graphe g et une constante k 
# k est la taille du sous arbre

# Dans le cas general, on pourrait verifier si le graphe est un arbre et executer cette fonction si c'est le cas

    g.show() # on visualise l'entree

    if not g.is_tree():
        print("Graph is not a tree")
    
    else:
    
        print("Generating linear program...")

        # constantes
        edges = g.edges(labels=False) # liste de couples representants les aretes
        n = g.order() # taille de l'arbre
        a = g.size() # nombre d'arete
        m = n - 1 # borne sup du degre max du graphe (a paufiner et renommer) 
            
        p = MixedIntegerLinearProgram() # tester des solvers 
        
        # variables du PL
        x = p.new_variable(binary = True) # aretes, x[e] si e arete presente dans le sous-arbre selectionne 
        y = p.new_variable(binary = True) # sommets y[i] si i est present dans le sous-arbre
        f = p.new_variable(binary = True) # feuilles f[i] si i est une feuille du sous-arbre
        
        # fonction objectif
        p.set_objective(p.sum(f[i] for i in range(n))) # + p.sum(x[e] for e in edges) )
        
        # contraintes
        p.add_constraint(p.sum(y[i] for i in range(n)) == k)  # k la taille du sous arbre 
        p.add_constraint(p.sum(x[e] for e in edges) == k - 1)  # arbre connexe (m = n + 1)

        # contraintes aretes (generer)
        for i,j in edges:
            p.add_constraint(x[i, j] <= (y[i] + y[j])/2)  # presence d'une arete

        for i in range(n): # parcours des sommets pour determiner les feuilles
            p.add_constraint(f[i] <= y[i]) # une feuille est un sommet 

            degree = p.sum( (x[i,j] + x[j,i]) for j in g.neighbors(i)) # degre du sommet i
                
            p.add_constraint(f[i] <= 1 + (1./m) - (degree * (1./m) ) )  # contraintes sur les feuilles
        
        return p

In [ ]:
def solve_program(p, g):        
        # resolution
        print("Solving...")
        
        try:
            print(p.solve())
            print("Aretes x")
            print(p.get_values(x))
            print("Sommets y")
            print(p.get_values(y))
            print("Feuilles f")
            print(p.get_values(f))

            # affichage de la solution
            
            #liste de sommets du sous-arbre induit
            ve = []
            for k, v in p.get_values(y).iteritems():
                if v == 1:
                    ve.append(k)

            #liste d'aretes du sous-arbre induit
            ed = []
            for k, v in p.get_values(x).iteritems():
                if v == 1:
                    ed.append(k)

            #sous-graphe avec les sommets et aretes choisies
            g2 = g.subgraph(vertices=ve, edges=ed)
            g2.show()
            print("Solved.")
        
        except:
            print("Impossible to solve.")

In [ ]:
g = graphs.FibonacciTree(7)
k = 12
p = generate_program(g, k)
solve_program(p, g)        